# SQL Extra Topics - SQLAlchemy

The SQLAlchemy SQL Toolkit and Object Relational Mapper is a comprehensive set of tools for working with databases and Python

We'll need:

- [Azure SQL Database](https://portal.azure.com/)

- [ODBC Driver for SQL Server](https://docs.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver15) #Necesitamos un driver. Aquí se descarga.  

- [pyodbc](https://pypi.org/project/pyodbc/)#Modulo de python necesario. 

- [SQLAlchemy](https://docs.sqlalchemy.org/en/13/intro.html)#instalamos la libreria. 

In [1]:
# imports

import os
from dotenv import load_dotenv#Instalar.
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pandas as pd

sqlalchemy.__version__

ModuleNotFoundError: No module named 'dotenv'

---

### dotenv

https://pypi.org/project/python-dotenv/

In [3]:
load_dotenv('.env')#sirven oara generar variables de entorno que estan en un documento doc. y el ".env" estan configurados para ser ignorados. 

DATABASE_PASSWORD = os.environ.get("DATABASE_PASSWORD")#en esta variable carga la passwrod
DATABASE_PASSWORD#ojo, que asi la revelas, no imprimirla, solo meterla en la varibale. 

NameError: name 'load_dotenv' is not defined

---

In [2]:
# connection string params => https://docs.sqlalchemy.org/en/14/dialects/mssql.html#pass-through-exact-pyodbc-string

driver = 'Driver={ODBC Driver 17 for SQL Server};'#Ver celda de abajo. 
server = 'Server=tcp:sqlironhack.database.windows.net,1433;'
database = 'Database=AdventureWorksLT;'
uid = 'Uid=azureuser;'
pwd = f'Pwd={DATABASE_PASSWORD};'
config = 'Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'

NameError: name 'DATABASE_PASSWORD' is not defined

In [ ]:
# connection string assembly #En settings de azure "Connection string". ODBC. #Poner que el driver es el 17(Ver variable driver). Nosotros instalamos el 17.

connection_string = driver+server+database+uid+pwd+config 
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})#al argumento query le metemos un diccionario. El primero es el motor de conexion mas el driver. 
connection_url

In [ ]:
# SQL Engine => dialect+driver (https://docs.sqlalchemy.org/en/14/core/engines.html)

engine = create_engine(connection_url)
type(engine)#Objeto sql. 

---

In [ ]:
# Pandas!!!

df = pd.read_sql_query("SELECT * FROM SalesLT.ProductModel", engine)#primero la query, y segundo el motor. 
df

---

### Let's see some examples...

In [ ]:
query_1 = '''
SELECT TOP 20 pc.Name as CategoryName, p.name as ProductName 
FROM SalesLT.ProductCategory pc JOIN SalesLT.Product p ON pc.productcategoryid = p.productcategoryid
ORDER BY CategoryName
'''

In [ ]:
df_q1 = pd.read_sql_query(query_1, engine)
df_q1

In [ ]:
query_2 = '''
SELECT c.FirstName + ' ' + c.LastName AS [Customer Fullname], p.Name AS [Product Name]
FROM SalesLT.Customer AS c
    INNER JOIN SalesLT.SalesOrderHeader AS soh ON c.CustomerID = soh.CustomerID
    INNER JOIN SalesLT.SalesOrderDetail AS shd ON soh.SalesOrderID = shd.SalesOrderID
    INNER JOIN SalesLT.Product AS p ON shd.ProductID = p.ProductID
ORDER BY [Customer Fullname], [Product Name]
'''

In [ ]:
df_q2 = pd.read_sql_query(query_2, engine)
df_q2

In [ ]:
query_3 = '''
SELECT   p.name, COUNT(*) AS 'Total Orders'
FROM SalesLT.Product AS p
    INNER JOIN SalesLT.SalesOrderDetail AS sod ON p.ProductID = sod.ProductID
GROUP BY  p.Name
ORDER BY 'Total Orders' DESC
'''

In [ ]:
df_q3 = pd.read_sql_query(query_3, engine)
df_q3

---